<a href="https://colab.research.google.com/github/mahmud-nobe/Cluster-Membership/blob/master/cluster_membership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import packages
import pandas as pd
import pandas_profiling as pd_prof
import numpy as np
import matplotlib.pyplot as plt

## Use random forest to create and evaluate new model
from sklearn.ensemble import RandomForestRegressor

In [3]:

# import member dataset
member = pd.read_csv('https://raw.githubusercontent.com/mahmud-nobe/Cluster-Membership/master/n884_mem.csv')
member.describe()

,RA_ICRS,DE_ICRS,Source,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP,PMemb,_RA.icrs,_DE.icrs
count,1107.000000,1107.000000,1.107000e+03,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1098.000000,1107.000000,1107.000000,1107.000000
mean,35.669128,57.136607,4.584198e+17,135.099489,-3.563817,0.394660,0.055318,-0.589288,0.103309,-1.035505,0.115290,266.095754,15.212058,1.032836,0.488889,35.669133,57.136611
std,0.203451,0.112922,2.951795e+13,0.110841,0.112720,0.050145,0.030342,0.217097,0.057488,0.219616,0.062315,23.490328,1.849833,0.361234,0.245931,0.203451,0.112922
min,35.343833,56.821045,4.583058e+17,134.923423,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563,0.100000,35.343838,56.821049
25%,35.508695,57.068122,4.584059e+17,135.013822,-3.635055,0.358750,0.034250,-0.712000,0.064000,-1.164500,0.073500,253.000000,14.246502,0.726127,0.300000,35.508699,57.068126
50%,35.624218,57.139308,4.584115e+17,135.076991,-3.565761,0.394100,0.045800,-0.586000,0.085000,-1.049000,0.096000,268.000000,15.608300,1.020511,0.500000,35.624224,57.139313
75%,35.824777,57.209976,4.584531e+17,135.166715,-3.490694,0.431800,0.066300,-0.471500,0.123000,-0.930500,0.138500,282.000000,16.475282,1.306833,0.700000,35.824782,57.209982
max,36.154135,57.398243,4.584645e+17,135.382923,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.448000,327.000000,17.996572,2.581698,1.000000,36.154139,57.398246


In [4]:
member.head()

,RA_ICRS,DE_ICRS,Source,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP,PMemb,Cluster,_RA.icrs,_DE.icrs
0,35.414700,56.998612,458311532056974336,135.016952,-3.741123,0.4406,0.0304,-0.340,0.061,-0.800,0.066,258,14.762490,0.658170,0.5,NGC_884,35.414702,56.998615
1,35.417161,56.998057,458311532056974464,135.018405,-3.741186,0.4387,0.0980,-0.303,0.190,-0.970,0.197,280,17.157730,1.647366,0.3,NGC_884,35.417163,56.998061
2,35.392245,56.996236,458311566416713600,135.006249,-3.747538,0.4182,0.0435,-0.746,0.084,-1.338,0.093,270,15.710485,0.956811,0.7,NGC_884,35.392251,56.996242
3,35.388396,56.992285,458311566416713984,135.005627,-3.751968,0.3541,0.0242,-0.688,0.055,-1.219,0.053,253,13.630529,0.637017,0.4,NGC_884,35.388402,56.992290
4,35.409224,57.006802,458311600776450816,135.011336,-3.734447,0.3837,0.0590,-0.453,0.107,-1.159,0.116,273,16.240725,1.258944,0.2,NGC_884,35.409227,57.006807


In [5]:
pd_prof.ProfileReport(member)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,18
Number of observations,1107
Total Missing (%),0.0%
Total size in memory,155.8 KiB
Average record size in memory,144.1 B
Numeric,11
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,7


In [13]:
features = member.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13]]
features

,RA_ICRS,DE_ICRS,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP
0,35.414700,56.998612,135.016952,-3.741123,0.4406,0.0304,-0.340,0.061,-0.800,0.066,258,14.762490,0.658170
1,35.417161,56.998057,135.018405,-3.741186,0.4387,0.0980,-0.303,0.190,-0.970,0.197,280,17.157730,1.647366
2,35.392245,56.996236,135.006249,-3.747538,0.4182,0.0435,-0.746,0.084,-1.338,0.093,270,15.710485,0.956811
3,35.388396,56.992285,135.005627,-3.751968,0.3541,0.0242,-0.688,0.055,-1.219,0.053,253,13.630529,0.637017
4,35.409224,57.006802,135.011336,-3.734447,0.3837,0.0590,-0.453,0.107,-1.159,0.116,273,16.240725,1.258944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,35.475146,57.108823,135.010074,-3.626319,0.4078,0.1628,-0.069,0.273,-1.517,0.295,284,17.955290,1.558842
1103,35.820625,57.127543,135.179961,-3.543967,0.4018,0.0393,-0.490,0.070,-0.933,0.081,255,15.145645,0.795093
1104,35.504081,57.113965,135.023094,-3.616097,0.3138,0.0682,-1.096,0.125,-0.913,0.136,245,16.569098,1.390498
1105,35.875661,57.121199,135.210233,-3.539511,0.4552,0.0455,-0.395,0.085,-0.938,0.100,246,15.643492,0.929671


In [14]:
features.describe()

,RA_ICRS,DE_ICRS,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP
count,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1098.000000
mean,35.669128,57.136607,135.099489,-3.563817,0.394660,0.055318,-0.589288,0.103309,-1.035505,0.115290,266.095754,15.212058,1.032836
std,0.203451,0.112922,0.110841,0.112720,0.050145,0.030342,0.217097,0.057488,0.219616,0.062315,23.490328,1.849833,0.361234
min,35.343833,56.821045,134.923423,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563
25%,35.508695,57.068122,135.013822,-3.635055,0.358750,0.034250,-0.712000,0.064000,-1.164500,0.073500,253.000000,14.246502,0.726127
50%,35.624218,57.139308,135.076991,-3.565761,0.394100,0.045800,-0.586000,0.085000,-1.049000,0.096000,268.000000,15.608300,1.020511
75%,35.824777,57.209976,135.166715,-3.490694,0.431800,0.066300,-0.471500,0.123000,-0.930500,0.138500,282.000000,16.475282,1.306833
max,36.154135,57.398243,135.382923,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.448000,327.000000,17.996572,2.581698


In [7]:
targets = member['PMemb']
targets

0       0.5
1       0.3
2       0.7
3       0.4
4       0.2
       ... 
1102    0.1
1103    0.6
1104    0.2
1105    0.3
1106    0.6
Name: PMemb, Length: 1107, dtype: float64

In [15]:
member.dtypes

RA_ICRS     float64
DE_ICRS     float64
Source        int64
GLON        float64
GLAT        float64
plx         float64
e_plx       float64
pmRA        float64
e_pmRA      float64
pmDE        float64
e_pmDE      float64
o_Gmag        int64
Gmag        float64
BP-RP       float64
PMemb       float64
Cluster      object
_RA.icrs    float64
_DE.icrs    float64
dtype: object

ValueError: ignored

In [20]:
from numpy import float32
features = features.astype('float32')
targets = targets.astype('float32')
features.dtypes

RA_ICRS    float32
DE_ICRS    float32
GLON       float32
GLAT       float32
plx        float32
e_plx      float32
pmRA       float32
e_pmRA     float32
pmDE       float32
e_pmDE     float32
o_Gmag     float32
Gmag       float32
BP-RP      float32
dtype: object

In [17]:
features.describe()

,RA_ICRS,DE_ICRS,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP
count,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1098.000000
mean,35.669128,57.136600,135.099487,-3.563817,0.394660,0.055318,-0.589288,0.103309,-1.035505,0.115290,266.095764,15.212061,1.032835
std,0.203451,0.112922,0.110841,0.112720,0.050145,0.030342,0.217097,0.057488,0.219616,0.062315,23.490328,1.849833,0.361234
min,35.343834,56.821045,134.923416,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563
25%,35.508694,57.068121,135.013824,-3.635055,0.358750,0.034250,-0.712000,0.064000,-1.164500,0.073500,253.000000,14.246501,0.726127
50%,35.624218,57.139309,135.076996,-3.565761,0.394100,0.045800,-0.586000,0.085000,-1.049000,0.096000,268.000000,15.608300,1.020511
75%,35.824778,57.209978,135.166718,-3.490694,0.431800,0.066300,-0.471500,0.123000,-0.930500,0.138500,282.000000,16.475282,1.306833
max,36.154137,57.398243,135.382919,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.448000,327.000000,17.996572,2.581698


In [24]:
# Use Random Forest on whole dataset and calculate the R^2 value 
# considering the out of bag (OOB) samples
rfr = RandomForestRegressor(n_estimators = 100, oob_score = True)
rfr.fit(features, targets)


ValueError: ignored

In [23]:
rfr.score(features, targets)

NotFittedError: ignored